In [1]:
import tsraster.prep  as tr
import numpy as np
from scipy.stats import linregress
import rasterio
import copy

In [2]:
stackedRaster = tr.image_to_array("C:/Users/Python3/Documents/wildfire_FRAP/Data/Actual/ICLUS/iclus_V2.0/iclus_RAW/Clipped/")
print(stackedRaster.shape)
stackedRaster[stackedRaster == 2147483647] = 0
#print(stackedRaster)

(1209, 941, 20)


In [3]:
slopeRaster = np.zeros((1209, 941))
print(slopeRaster.shape)

(1209, 941)


In [4]:

import statsmodels.formula.api as smf

Years = np.arange(2005.0, 2105.0, 5.0)

for x in range(0, 1209):
    iterRow = stackedRaster[x, :, :]
    for y in range(0, 941):
        timeSeries = iterRow[y, :]
       
        #results = smf.OLS(timeSeries, Years).fit()
        results = linregress(timeSeries, Years)
        
        slopeRaster[x][y] = results.slope
        
slopeRaster = np.nan_to_num(slopeRaster)

C:\Users\Python3\Anaconda3\envs\ts-raster\lib\site-packages\scipy\stats\_stats_mstats_common.py:107: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
C:\Users\Python3\Anaconda3\envs\ts-raster\lib\site-packages\scipy\stats\_stats_mstats_common.py:119: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


In [5]:
timeSeries

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [6]:

with rasterio.open("../Data/Examples/3month_ts/aet/aet-201201.tif") as exampleRast:
        array = exampleRast.read()
        profile = exampleRast.profile
        profile.update(dtype=rasterio.float32, count=1, compress='lzw',nodata=0)

slopeRaster = np.float32(slopeRaster)      
        
with rasterio.open("../Data/Actual/ICLUS\iclus_V2.0\iclus_RAW\Slope\Slope_Initial.tif", 'w', **profile) as exampleRast:
    exampleRast.write(slopeRaster, 1)

In [7]:
#read in 2005 Raster as baseline for historical regressions

raster_2005 = tr.read_images("C:/Users/Python3/Documents/wildfire_FRAP/Data/Actual/ICLUS/iclus_V2.0/iclus_RAW/Clipped/bhd_2005_Clip.tif")
raster_2005 = raster_2005[0].ReadAsArray()
raster_2005[raster_2005 == 2147483647] = 0
raster_2005.shape

(1209, 941)

In [8]:
###Linear Regressions to back-calculate housing density from slope

newYearList = list(range(1970, 2004))



for x in range(1970, 2005):
    yearDiff = x - 2005
    iterRaster = raster_2005 + (slopeRaster * yearDiff)
    iterRaster[iterRaster < 0] = 0
    iterRaster = np.float32(iterRaster) 
    
    with rasterio.open("../Data/Actual/ICLUS/iclus_V2.0/iclus_RAW/SimpleRegs/bhd_" + str(x) + "linreg.tif", 'w', **profile) as exampleRast:
        exampleRast.write(iterRaster, 1)

In [22]:
###Interpolated linear regression
Years = np.arange(2005, 2100, 5)

for x in range(len(Years)):
    earlyYear = Years[x]
    lateYear = Years[x]+ 5
    for y in range(0, 5):
        iterYear = earlyYear + y
        
        earlyRaster = tr.read_images("C:/Users/Python3/Documents/wildfire_FRAP/Data/Actual/ICLUS/iclus_V2.0/iclus_RAW/Clipped/bhd_" + str(earlyYear) + "_Clip.tif")
        earlyRaster = earlyRaster[0].ReadAsArray()
        earlyRaster[earlyRaster == 2147483647] = 0
        
        lateRaster = tr.read_images("C:/Users/Python3/Documents/wildfire_FRAP/Data/Actual/ICLUS/iclus_V2.0/iclus_RAW/Clipped/bhd_" + str(lateYear) + "_Clip.tif")
        lateRaster = lateRaster[0].ReadAsArray()
        lateRaster[lateRaster == 2147483647] = 0
        
        #calculate anual raster by weighting between prior and successive raster
        iterRaster = ((earlyRaster * (5-y)) + (lateRaster * (0+y)))/5
        iterRaster = np.float32(iterRaster)
        
        with rasterio.open("../Data/Actual/ICLUS/iclus_V2.0/iclus_RAW/SimpleRegs/bhd_" + str(iterYear) + "linreg.tif", 'w', **profile) as exampleRast:
            exampleRast.write(iterRaster, 1)
        
        


In [19]:
x

18

In [20]:
Years[x]

2095

In [21]:
lateYear


2100